In [2]:
import networkx as nx
from tqdm import tqdm
import random
from leafer import Leafer
import numpy as np
import pickle
import os
import glob
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn


seed = 42
random.seed(seed)
np.random.seed(seed)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
G = nx.DiGraph()
for synset in (wn.all_synsets('n')):
    name = synset.name()
    G.add_node(name)
    hyponyms = synset.hyponyms()

    for hypo in hyponyms:
        new_name = hypo.name()
        G.add_node(new_name)
        G.add_edge(name, new_name)

for synset in (wn.all_synsets('v')):
    name = synset.name()
    G.add_node(name)
    hyponyms = synset.hyponyms()

    for hypo in hyponyms:
        new_name = hypo.name()
        G.add_node(new_name)
        G.add_edge(name, new_name)


In [4]:
while True:
    try:
        cycle = nx.find_cycle(G)
        print(cycle)
        G.remove_edge(*cycle[0])
    except:
        break

[('restrain.v.01', 'inhibit.v.04'), ('inhibit.v.04', 'restrain.v.01')]


In [5]:
l = Leafer(G)


In [9]:
p_case = {
    'only_leafs': 0,
    'only_leafs_divided': 0,
    'not_only_leafs': 0.05,
    'parents': 0.00001,
    'simple_triplet_2parent': 0.1,
    'simple_triplet_grandparent': 0,
    'only_child_leaf': 0
}

train, test = l.split_train_test(
    generation_depth=0,  # до какого уровня в топ. сортировке идти
    p=p_case,  # вероятность что подходящий случай уйдет в тест
    p_divide_leafs=0,
    # вероятность что листья поделим пополам трейн-тест
    # а не засунем целый случай в трейн или в тест
    min_to_test_rate=1,
    # минимальное количество доли вершин которых не было в
    # трейне чтобы поделить пополам на трейн-тест
    # то есть если 6\10 вершин были трейне то значит все 10 в трейн
    # если 5\10 были в трейне, то значит оставшиеся можем кинуть в тест
    weights=[0.00, 0.0, 0.1, 0.1, 0.00, 0.8],
   # weights=[0.00, 0.0, 0., 0.0, 0.00, 1.],

    # веса в соответствии
    # один ребенок, только листья, не только листья
    # триплеты с 2 родителями, триплеты такие что мидл нода имеет
    # 1 ребенка, предсказание родителя
    #p_parent=1
)

predict_hypernym 43850 43850
leafs_and_no_leafs 4792 4792
simple_triplet_2parent 1367 1367
leafs_and_no_leafs 204 204
predict_hypernym 828 828
simple_triplet_2parent 170 170


In [10]:
def add_definitions(elem):
    if elem['case'] == 'predict_hypernym':
        elem['child_def'] = wn.synset(elem['children'].replace(' ', '_')).definition()
      #  elem['parent_def'] = wn.synset(elem['parents']).definition()
    elif elem['case'] == 'predict_multiple_hypernyms':
        elem['child_def'] = wn.synset(elem['children'].replace(' ', '_')).definition()

    elif elem['case'] == 'simple_triplet_grandparent':
        elem['child_def'] = wn.synset(elem['children'].replace(' ', '_')).definition()
        elem['grandparent_def'] = wn.synset(elem['grandparents']).definition()
    elif elem['case'] == 'only_child_leaf':
        elem['grandparent_def'] = wn.synset(elem['grandparents']).definition()
        elem['parent_def'] = wn.synset(elem['parents']).definition()
    elif elem['case'] == 'simple_triplet_2parent':
        elem['1parent_def'] = wn.synset(elem['parents'][0]).definition()
        elem['2parent_def'] = wn.synset(elem['parents'][1]).definition()
    else:
        elem['parent_def'] = wn.synset(elem['parents']).definition()

In [11]:
for k in range(3):
    for i, elem in enumerate(train):
        try:
            add_definitions(elem)
        except:
            print(k, i, elem)
            train.remove(elem)

In [12]:
counter = 0
for k in range(3):

    for i, elem in enumerate(test):
        try:
            add_definitions(elem)
        except:
            print(k, i, elem)
            counter += 1
            test.remove(elem)

print(counter)

0


In [15]:
train_out = '/home/data/v.moskvoretskii/LLM_Taxonomy/wnet/taxollama3_train.pickle'
test_out = '/home/data/v.moskvoretskii/LLM_Taxonomy/taxollama3_test.pickle'

with open(train_out, 'wb') as f:
    pickle.dump(train, f)

with open(test_out, 'wb') as f:
    pickle.dump(test, f)